In [ ]:
##ChatGPT認為可以改成的py版本

from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
import os
import configparser

app = Flask(__name__)

# Load config file
config = configparser.ConfigParser()
config.read('config.ini')

# Line bot config
line_bot_api = LineBotApi(config['line_bot']['channel_access_token'])
handler = WebhookHandler(config['line_bot']['channel_secret'])

# Azure Text Analytics config
endpoint = config['text_analytics']['endpoint']
api_key = config['text_analytics']['api_key']
credentials = AzureKeyCredential(api_key)
text_analytics_client = TextAnalyticsClient(endpoint=endpoint, credential=credentials)

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_text_message(event):
    text = event.message.text
    documents = [text]
    result = text_analytics_client.analyze_sentiment(documents)
    sentiment = result[0].sentiment
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=sentiment)
    )

if __name__ == "__main__":
    port = int(os.environ.get('PORT', 5000))
    app.run(host='0.0.0.0', port=port)
